# Ontotext GraphDB QA Chain

This notebook shows how to use LLMs to provide natural language querying (NLQ to SPARQL, also called text2sparql) for [Ontotext GraphDB](https://graphdb.ontotext.com/). Ontotext GraphDB is a graph database and knowledge discovery tool compliant with [RDF](https://www.w3.org/RDF/) and [SPARQL](https://www.w3.org/TR/sparql11-query/).

## GraphDB LLM Functionalities

GraphDB supports some LLM integration functionalities as described in [https://github.com/w3c/sparql-dev/issues/193](https://github.com/w3c/sparql-dev/issues/193):

[gpt-queries](https://graphdb.ontotext.com/documentation/10.5/gpt-queries.html)

* magic predicates to ask an LLM for text, list or table using data from your knowledge graph (KG)
* query explanation
* result explanation, summarization, rephrasing, translation

[retrieval-graphdb-connector](https://graphdb.ontotext.com/documentation/10.5/retrieval-graphdb-connector.html)

* Indexing of KG entities in a vector database
* Supports any text embedding algorithm and vector database
* Uses the same powerful connector (indexing) language that GraphDB uses for Elastic, Solr, Lucene
* Automatic synchronization of changes in RDF data to the KG entity index
* Supports nested objects (no UI support in GraphDB version 10.5)
* Serializes KG entities to text like this (e.g. for a Wines dataset):

```
Franvino:
- is a RedWine.
- made from grape Merlo.
- made from grape Cabernet Franc.
- has sugar dry.
- has year 2012.
```

[talk-to-graph](https://graphdb.ontotext.com/documentation/10.5/talk-to-graph.html)

* A simple chatbot using a defined KG entity index

## Querying the GraphDB Database

For this tutorial, we won't use the GraphDB LLM integration, but SPARQL generation from NLQ. We'll use the Star Wars API (SWAPI) ontology and dataset that you can examine [here](https://drive.google.com/file/d/1wQ2K4uZp4eq3wlJ6_F_TxkOolaiczdYp/view?usp=drive_link).

You will need to have a running GraphDB instance. This tutorial shows how to run the database locally using the [GraphDB Docker image](https://hub.docker.com/r/ontotext/graphdb). It provides a docker compose set-up, which populates GraphDB with the Star Wars dataset. All nessessary files including this notebook can be downloaded from GDrive.

### Set-up

* Install [Docker](https://docs.docker.com/get-docker/). This tutorial is created using Docker version `24.0.7` which bundles [Docker Compose](https://docs.docker.com/compose/). For earlier Docker versions you may need to install Docker Compose separately.
* Download all files from [GDrive](https://drive.google.com/drive/folders/18dN7WQxfGu26Z9C9HUU5jBwDuPnVTLbl) in a local folder on your machine.
* Start GraphDB with the following script executed from this folder
  ```
  docker build --tag graphdb .
  docker compose up -d graphdb
  ```
  You need to wait a couple of seconds for the database to start on `http://localhost:7200/`. The Star Wars dataset `starwars-data.trig` is automatically loaded into the `langchain` repository. The local SPARQL endpoint `http://localhost:7200/repositories/langchain` can be used to run queries against. You can also open the GraphDB Workbench from your favourite web browser `http://localhost:7200/sparql` where you can make queries interactively.
* Working environment

If you use `conda`, create and activate a new conda env (e.g. `conda create -n graph_ontotext_graphdb_qa python=3.9.18`).
Install the following libraries:

```
pip install jupyter==1.0.0
pip install openai==1.6.1
pip install rdflib==7.0.0
pip install langchain-openai==0.0.2
pip install langchain
```

Run Jupyter with
```
jupyter notebook
```

### Specifying the Ontology

In order for the LLM to be able to generate SPARQL, it needs to know the knowledge graph schema (the ontology). It can be provided using one of two parameters on the `OntotextGraphDBGraph` class:

* `query_ontology`: a `CONSTRUCT` query that is executed on the SPARQL endpoint and returns the KG schema statements. We recommend that you store the ontology in its own named graph, which will make it easier to get only the relevant statements (as the example below). `DESCRIBE` queries are not supported, because `DESCRIBE` returns the Symmetric Concise Bounded Description (SCBD), i.e. also the incoming class links. In case of large graphs with a million of instances, this is not efficient. Check https://github.com/eclipse-rdf4j/rdf4j/issues/4857
* `local_file`: a local RDF ontology file. Supported RDF formats are `Turtle`, `RDF/XML`, `JSON-LD`, `N-Triples`, `Notation-3`, `Trig`, `Trix`, `N-Quads`.

In either case, the ontology dump should:

* Include enough information about classes, properties, property attachment to classes (using rdfs:domain, schema:domainIncludes or OWL restrictions), and taxonomies (important individuals).
* Not include overly verbose and irrelevant definitions and examples that do not help SPARQL construction.

In [1]:
from langchain_community.graphs import OntotextGraphDBGraph

# feeding the schema using a user construct query

graph = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/langchain",
    query_ontology="CONSTRUCT {?s ?p ?o} FROM <https://swapi.co/ontology/> WHERE {?s ?p ?o}",
)

In [4]:
%ls

 Volume in drive D is DRIVE
 Volume Serial Number is 64AC-E066

 Directory of D:\Development\Study\langchain-ontotext

06/02/2024  02:19 pm    <DIR>          .
06/02/2024  02:05 pm    <DIR>          ..
06/02/2024  02:17 pm    <DIR>          .ipynb_checkpoints
06/02/2024  02:05 pm             1,787 config.ttl
06/02/2024  02:05 pm               116 docker-compose.yaml
06/02/2024  02:05 pm               247 Dockerfile
06/02/2024  02:19 pm            20,734 graph_ontotext_graphdb_qa.ipynb
06/02/2024  02:05 pm               980 graphdb_create.sh
06/02/2024  02:05 pm             3,811 rdfs.ttl
06/02/2024  02:05 pm         2,219,534 starwars-data.trig
06/02/2024  02:05 pm            99,987 starwars-ontology.nt
               8 File(s)      2,347,196 bytes
               3 Dir(s)  684,690,591,744 bytes free


In [5]:
# feeding the schema using a local RDF file

graph = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/langchain",
    local_file="starwars-ontology.nt",  # change the path here
)

Either way, the ontology (schema) is fed to the LLM as `Turtle` since `Turtle` with appropriate prefixes is most compact and easiest for the LLM to remember.

The Star Wars ontology is a bit unusual in that it includes a lot of specific triples about classes, e.g. that the species `:Aleena` live on `<planet/38>`, they are a subclass of `:Reptile`, have certain typical characteristics (average height, average lifespan, skinColor), and specific individuals (characters) are representatives of that class:


```
@prefix : <https://swapi.co/vocabulary/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Aleena a owl:Class, :Species ;
    rdfs:label "Aleena" ;
    rdfs:isDefinedBy <https://swapi.co/ontology/> ;
    rdfs:subClassOf :Reptile, :Sentient ;
    :averageHeight 80.0 ;
    :averageLifespan "79" ;
    :character <https://swapi.co/resource/aleena/47> ;
    :film <https://swapi.co/resource/film/4> ;
    :language "Aleena" ;
    :planet <https://swapi.co/resource/planet/38> ;
    :skinColor "blue", "gray" .

    ...

 ```


In order to keep this tutorial simple, we use un-secured GraphDB. If GraphDB is secured, you should set the environment variables 'GRAPHDB_USERNAME' and 'GRAPHDB_PASSWORD' before the initialization of `OntotextGraphDBGraph`.

```python
os.environ["GRAPHDB_USERNAME"] = "graphdb-user"
os.environ["GRAPHDB_PASSWORD"] = "graphdb-password"

graph = OntotextGraphDBGraph(
    query_endpoint=...,
    query_ontology=...
)
```


### Question Answering against the StarWars Dataset

We can now use the `OntotextGraphDBQAChain` to ask some questions.

In [6]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from langchain.chains import OntotextGraphDBQAChain
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

# We'll be using an OpenAI model which requires an OpenAI API Key.
# However, other models are available as well:
# https://python.langchain.com/docs/integrations/chat/

# Set the environment variable `OPENAI_API_KEY` to your OpenAI API key
# os.environ["OPENAI_API_KEY"] = "sk-***"

# Any available OpenAI model can be used here.
# We use 'gpt-4-1106-preview' because of the bigger context window.
# The 'gpt-4-1106-preview' model_name will deprecate in the future and will change to 'gpt-4-turbo' or similar,
# so be sure to consult with the OpenAI API https://platform.openai.com/docs/models for the correct naming.

chain = OntotextGraphDBQAChain.from_llm(
    ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview"),
    graph=graph,
    verbose=True,
)

Let's ask a simple one.

In [8]:
chain.invoke({chain.input_key: "What is the climate on Tatooine?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>

SELECT ?climate
WHERE {
  ?planet ns1:climate ?climate ;
          rdfs:label "Tatooine" .
}

> Finished chain.


'The climate on Tatooine is arid.'

In [9]:
chain.invoke({chain.input_key: "What is the climate on Naboo?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>

SELECT ?climate
WHERE {
  ?planet ns1:climate ?climate ;
          rdfs:label "Naboo" .
}

> Finished chain.


'The climate on Naboo is temperate.'

In [19]:
chain.invoke({chain.input_key: "Who is taller, Luke or Han Solo?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>

SELECT ?characterName (MAX(?height) AS ?maxHeight)
WHERE {
  ?character ns1:height ?height ;
             ns1:name ?characterName .
  FILTER (?characterName IN ("Luke Skywalker", "Han Solo"))
}
GROUP BY ?characterName
ORDER BY DESC(?maxHeight)
LIMIT 1

> Finished chain.


'Based on the information provided, I do not have the data to determine who is taller between Luke and Han Solo. If you can provide the heights or any relevant details from a SPARQL query or another source, I would be happy to assist you with an answer.'

In [16]:
chain.invoke({chain.input_key: "Describe luke skywalker and han solo. From here, who is taller?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?characterName ?height
WHERE {
  ?character rdfs:label ?characterName ;
             ns1:height ?height .
  FILTER (?characterName IN ("Luke Skywalker", "Han Solo"))
}
ORDER BY DESC(?height)

> Finished chain.


'Based on the information provided, Han Solo is taller than Luke Skywalker. Han Solo has a height of 180.0 units, while Luke Skywalker has a height of 172.0 units.'

In [18]:
chain.invoke({chain.input_key: "Describe luke skywalker."})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>

SELECT ?property ?value
WHERE {
  ?character rdfs:label "Luke Skywalker" .
  ?character ?property ?value .
}

> Finished chain.


'Luke Skywalker is a fictional character and the main protagonist of the original film trilogy of the Star Wars franchise created by George Lucas. Portrayed by Mark Hamill, Luke is an important figure in the Rebel Alliance\'s struggle against the Galactic Empire. He is the twin brother of Rebellion leader Princess Leia Organa of Alderaan, a friend and brother-in-law of smuggler Han Solo, and an apprentice to Jedi Masters Obi-Wan "Ben" Kenobi and Yoda. Luke is the son of fallen Jedi Anakin Skywalker (Darth Vader) and Queen of Naboo/Republic Senator Padmé Amidala, and the maternal uncle of Kylo Ren / Ben Solo.\n\nThe character has been depicted as a powerful Jedi Master in the now non-canon Star Wars expanded universe, where he is the husband of Mara Jade, the father of Ben Skywalker, and the maternal uncle of Jaina, Jacen, and Anakin Solo. In 2015, Luke Skywalker was selected by Empire magazine as the 50th greatest movie character of all time, and Fandomania.com ranked him at number 14 

And a bit more complicated one.

In [10]:
chain.invoke({chain.input_key: "What is the climate on Luke Skywalker's home planet?"})[
    chain.output_key
]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?climate
WHERE {
  ?character rdfs:label "Luke Skywalker" .
  ?character ns1:homeworld ?planet .
  ?planet ns1:climate ?climate .
}

> Finished chain.


"The climate on Luke Skywalker's home planet is arid."

We can also ask more complicated questions like

In [11]:
chain.invoke(
    {
        chain.input_key: "What is the average box office revenue for all the Star Wars movies?"
    }
)[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX ns1: <https://swapi.co/vocabulary/>

SELECT (AVG(?boxOffice) AS ?averageBoxOfficeRevenue)
WHERE {
  ?film a ns1:Film .
  ?film ns1:boxOffice ?boxOffice .
}

> Finished chain.


'The average box office revenue for all the Star Wars movies is approximately 754.1 million dollars.'

### Chain Modifiers

The Ontotext GraphDB QA chain allows prompt refinement for further improvement of your QA chain and enhancing the overall user experience of your app.


#### "SPARQL Generation" Prompt

The prompt is used for the SPARQL query generation based on the user question and the KG schema.

- `sparql_generation_prompt`

    Default value:
  ````python
    GRAPHDB_SPARQL_GENERATION_TEMPLATE = """
    Write a SPARQL SELECT query for querying a graph database.
    The ontology schema delimited by triple backticks in Turtle format is:
    ```
    {schema}
    ```
    Use only the classes and properties provided in the schema to construct the SPARQL query.
    Do not use any classes or properties that are not explicitly provided in the SPARQL query.
    Include all necessary prefixes.
    Do not include any explanations or apologies in your responses.
    Do not wrap the query in backticks.
    Do not include any text except the SPARQL query generated.
    The question delimited by triple backticks is:
    ```
    {prompt}
    ```
    """
    GRAPHDB_SPARQL_GENERATION_PROMPT = PromptTemplate(
        input_variables=["schema", "prompt"],
        template=GRAPHDB_SPARQL_GENERATION_TEMPLATE,
    )
  ````

#### "SPARQL Fix" Prompt

Sometimes, the LLM may generate a SPARQL query with syntactic errors or missing prefixes, etc. The chain will try to amend this by prompting the LLM to correct it a certain number of times.

- `sparql_fix_prompt`

    Default value:
  ````python
    GRAPHDB_SPARQL_FIX_TEMPLATE = """
    This following SPARQL query delimited by triple backticks
    ```
    {generated_sparql}
    ```
    is not valid.
    The error delimited by triple backticks is
    ```
    {error_message}
    ```
    Give me a correct version of the SPARQL query.
    Do not change the logic of the query.
    Do not include any explanations or apologies in your responses.
    Do not wrap the query in backticks.
    Do not include any text except the SPARQL query generated.
    The ontology schema delimited by triple backticks in Turtle format is:
    ```
    {schema}
    ```
    """
    
    GRAPHDB_SPARQL_FIX_PROMPT = PromptTemplate(
        input_variables=["error_message", "generated_sparql", "schema"],
        template=GRAPHDB_SPARQL_FIX_TEMPLATE,
    )
  ````

- `max_fix_retries`
  
    Default value: `5`

#### "Answering" Prompt

The prompt is used for answering the question based on the results returned from the database and the initial user question. By default, the LLM is instructed to only use the information from the returned result(s). If the result set is empty, the LLM should inform that it can't answer the question.

- `qa_prompt`
  
  Default value:
  ````python
    GRAPHDB_QA_TEMPLATE = """Task: Generate a natural language response from the results of a SPARQL query.
    You are an assistant that creates well-written and human understandable answers.
    The information part contains the information provided, which you can use to construct an answer.
    The information provided is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
    Make your response sound like the information is coming from an AI assistant, but don't add any information.
    Don't use internal knowledge to answer the question, just say you don't know if no information is available.
    Information:
    {context}
    
    Question: {prompt}
    Helpful Answer:"""
    GRAPHDB_QA_PROMPT = PromptTemplate(
        input_variables=["context", "prompt"], template=GRAPHDB_QA_TEMPLATE
    )
  ````

Once you're finished playing with QA with GraphDB, you can shut down the Docker environment by running
``
docker compose down -v --remove-orphans
``
from the directory with the Docker compose file.